# ASKBOT using Transformer

This is the continuation of the previous version of ASKBOT using PandasAI but it generated a lot of problems beacuse we can't ask questions with an empty dataframe cause it causes exceeptinos and warnings.

This new version of ASKBOT will use pre-trained Transformer models 

### IMPORTS

In [1]:
# PyAudio
import pyaudio 
import wave
import audioop
import webrtcvad
# Speech2Text + Text2Speech
import speech_recognition as sr
import pyttsx3
# Transformer
from transformers import pipeline

### GLOBAL PARAMETERS

In [2]:
FORMAT = pyaudio.paInt16
CHANNELS = 1 
RATE = 16000
CHUNK = 320
THRESHOLD = 500
SILENCE_TIMEOUT = 1

### RECORD AUDIO

In [ ]:
audio = pyaudio.PyAudio()
stram = audio.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    frames_per_buffer=CHUNK,
)
print("You can talk now.\n\nRecording...")
#
frames = []
silence_counter = 0
vad = webrtcvad.Vad()
vad.set_mode(3) # <--- Aggressive detection to record voice

try:
    while True:
        data = stram.read(CHUNK)
        frames.append(data)
        is_speech = vad.is_speech(data, RATE)
        if not is_speech:
            silence_counter += 1
        else:
            silence_counter = 0
        if silence_counter > int(SILENCE_TIMEOUT * ( RATE / CHUNK ) ):
            break
except Exception as e:
    print(f"Exception catched: {e}")
    
print("\nEnd of Recording.")

# Stop stream
stram.stop_stream()
stram.close()

# Stop Pyaudio
audio.terminate()

# Save the recorded data as a WAV file
with wave.open("Audios/recorded.wav", "wb") as audio_recorded:
    audio_recorded.setnchannels(CHANNELS)
    audio_recorded.setsampwidth(audio.get_sample_size(FORMAT))
    audio_recorded.setframerate(RATE)
    audio_recorded.writeframes(b"".join(frames))

print("Audio saved in \'Audios/\' as \'recorded.wav\'")

### Speech2Text <--> Text2Speech functions

In [ ]:
r = sr.Recognizer()

def Text2Speech(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()
    return True

def Speech2Text(audioFile):
    with sr.AudioFile(audioFile) as audio:
        audioFile = r.record(audio)
        print("Audio loaded.")
        try:
            text = r.recognize_google(audioFile, language="es-ES")
            print(f"You said: {text}")
            return text
        except Exception as e:
            print(f"Excepcion catched: {e}")
            return e

#### Speech2Text

In [ ]:
text = Speech2Text("Audios/recorded.wav")
type(text) # <--- We check the type of our text

### Transformer model